<a href="https://colab.research.google.com/github/Emily-KD/DS200/blob/main/Lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#########################################################
# DS 200, SP 2022 semester
# python code related to Lab 07 if you choose to adapt it
# If you use this code, copy-paste it one block at a time
# and try to understand what each block accomplishes
#########################################################

In [ ]:
# Load needed python resources, including folium mapping module
from datascience import *
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
import pandas as pd
np.set_printoptions(threshold=50)
import folium

/usr/local/lib/python3.7/dist-packages/datascience/tables.py:17: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.7/dist-packages/datascience/util.py:10: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  after removing the cwd from sys.path.


In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read massive dataset.  Your file path may be different,
# depending on where you stored the data files for Lab 6.
file_path = 'drive/MyDrive/DS200/Lab/Lab6/data/'
flights = Table.read_table(file_path + 'flights.csv')

In [ ]:
# Read airports dataset containing airport names and lat/lon coordinates
airports = Table.read_table(file_path + 'airports.csv')

In [ ]:
# Read airlines dataset containing airline names
airlines = Table.read_table(file_path + 'airlines.csv')
# This Table is small enough to print the whole thing:
airlines.show()

IATA_CODE,AIRLINE
UA,United Air Lines Inc.
AA,American Airlines Inc.
US,US Airways Inc.
F9,Frontier Airlines Inc.
B6,JetBlue Airways
OO,Skywest Airlines Inc.
AS,Alaska Airlines Inc.
NK,Spirit Air Lines
WN,Southwest Airlines Co.
DL,Delta Air Lines Inc.


In [ ]:
# Check out the columns available in the flights dataset
# Think about which might be interesting to display on a map
flights.labels

('YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY')

In [ ]:
# As an illustration, let's consider all flights
AmFlights = (flights.where('AIRLINE', are.contained_in('AA MQ')))

In [ ]:
# Count American flights for each ORIGIN_AIRPORT to get
# an idea of how many airports these airlines serve
airport_counts = AmFlights.group('ORIGIN_AIRPORT').sort('count', descending=True)
airport_counts

ORIGIN_AIRPORT,count
DFW,42078
ORD,21098
MIA,11126
LAX,5922
LGA,5583
JFK,4160
DCA,2385
MCO,2036
LAS,1943
SFO,1940


In [ ]:
# Find average WEATHER_DELAY for each ORIGIN_AIRPORT and join lat/lon info
# so that we can create a map with airport locations
avg_weather_delays = (AmFlights
     .select('WEATHER_DELAY', 'ORIGIN_AIRPORT', 'CANCELLED')
     .group('ORIGIN_AIRPORT', np.nanmean)
     .relabeled('WEATHER_DELAY nanmean', 'AVG_WEATHER_DELAY')
     .relabeled('CANCELLED nanmean', 'CANCELLED_FLIGHTS')
     .join('ORIGIN_AIRPORT', airports, 'IATA_CODE') 
     .join('ORIGIN_AIRPORT', airport_counts, 'ORIGIN_AIRPORT')
     .select('ORIGIN_AIRPORT', 'AVG_WEATHER_DELAY', 'CANCELLED_FLIGHTS', 'AIRPORT', 'LATITUDE', 
                                                'LONGITUDE', 'count')
     .relabeled('AIRPORT', 'AIRPORT_NAME')
)
# Let's see the top 10 rows of this new Table
avg_weather_delays

ORIGIN_AIRPORT,AVG_WEATHER_DELAY,CANCELLED_FLIGHTS,AIRPORT_NAME,LATITUDE,LONGITUDE,count
ABI,5.56034,0.109434,Abilene Regional Airport,32.4113,-99.6819,530
ABQ,1.0625,0.0635359,Albuquerque International Sunport,35.0402,-106.609,362
ACT,0.523077,0.13354,Waco Regional Airport,31.6113,-97.2305,322
AEX,1.09804,0.097561,Alexandria International Airport,31.3274,-92.5486,205
ALO,0.85,0.132812,Waterloo Regional Airport,42.5571,-92.4003,128
AMA,1.91304,0.13834,Rick Husband Amarillo International Airport,35.2194,-101.706,506
ASE,11.7692,0.191176,Aspen-Pitkin County Airport,39.2232,-106.869,204
ATL,0.415493,0.0676998,Hartsfield-Jackson Atlanta International Airport,33.6404,-84.4269,1226
AUS,0.608889,0.0530669,Austin-Bergstrom International Airport,30.1945,-97.6699,1451
AZO,1.61111,0.125954,Kalamazoo/Battle Creek International Airport,42.2349,-85.5521,262


In [ ]:
# Create folium map with circles having radius proportional to air time
weather_delaysMap = folium.Map(location=[35, -100],
     tiles="OpenStreetMap", zoom_start=4,
     width='100%', height='100%')
# Add Circles one by one using info from avg_airtime Table
for i in np.arange(avg_weather_delays.num_rows):
     folium.Circle([avg_weather_delays.column('LATITUDE')[i],
     avg_weather_delays.column('LONGITUDE')[i]],
     radius = 250 * avg_weather_delays.column('AVG_WEATHER_DELAY')[i],
     fill=True).add_to(weather_delaysMap)
# Once Map object is created, output it:
weather_delaysMap

###Step 1.
This map shows how often the delays are caused by weather. 

###Step 2.
I thought that this map looks interseting becasue there are more delays on the East Coast than there is on the West Coat. But when I think about it, it makes sense because usually the weather is much more severe on the East Coast.

In [ ]:
# Create folium map with circles having radius proportional to air time
cancel_Map = folium.Map(location=[35, -100],
     tiles="OpenStreetMap", zoom_start=4,
     width='100%', height='100%')
# Add Circles one by one using info from avg_airtime Table
for i in np.arange(avg_weather_delays.num_rows):
     folium.Circle([avg_weather_delays.column('LATITUDE')[i],
     avg_weather_delays.column('LONGITUDE')[i]],
     radius = 250 * avg_weather_delays.column('CANCELLED_FLIGHTS')[i],
     fill=True).add_to(cancel_Map)
# Once Map object is created, output it:
cancel_Map

###Step 1.
This map shows how often the flights are canceled.

###Step 2.
I thought that this map looks interseting becasue I thought it might have connections with how often the delays caused by weather. It is not clear and is not the right database to use to see if there is a connection, but based on the two maps above, it seems like there is a connection to both data.